In [1]:
%pip install --quiet --user --upgrade langchain langchain-community langchain-chroma pypdf
%pip install -qU langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0

In [2]:
import getpass
import os

In [3]:
os.environ['USER_AGENT'] = 'myagent'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


# Create vector db

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
!pip install --no-user langchain-community

In [6]:
import sys
sys.path.append("/root/.local/lib/python3.11/site-packages")

# Kiểm tra xem langchain_community có import được không
import langchain_community
print("langchain_community is installed correctly.")

langchain_community is installed correctly.


In [7]:
!pip install --no-user --quiet langchain==0.3.25 langchain-community==0.3.24 langchain-huggingface==0.2.0 langchain-chroma==0.2.4 chromadb==1.0.10 pypdf==5.5.0 sentence-transformers underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [8]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import torch
import os
from underthesea import word_tokenize

# Tiền xử lý tiếng Việt
def preprocess_vietnamese_text(text):
    return word_tokenize(text, format="text")

# Khởi tạo embedding đa ngôn ngữ
langchain_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# Tạo thư mục
directory_path = "./data/dataset"
test_dir = "./data/test"
os.makedirs(directory_path, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Di chuyển file từ ./data/dataset sang ./data/test
required_files = ["long-q.txt", "long-a.txt", "dang-q.txt", "dang-a.txt"]
for file in required_files:
    source = os.path.join(directory_path, file)
    destination = os.path.join(test_dir, file)
    if os.path.exists(source):
        !mv {source} {destination}
        print(f"Đã di chuyển {file} từ {directory_path} sang {test_dir}")
    else:
        raise FileNotFoundError(f"File {file} không tồn tại trong {directory_path}. Vui lòng kiểm tra.")

# Kiểm tra file trong ./data/test
for file in required_files:
    file_path = os.path.join(test_dir, file)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file} không tồn tại trong {test_dir}. Vui lòng kiểm tra.")

# Tạo tài liệu từ long-a.txt và dang-a.txt
docs = []
for answer_file in ["long-a.txt", "dang-a.txt"]:
    with open(os.path.join(test_dir, answer_file), "r", encoding="utf-8") as f:
        answers = f.readlines()
        for i, answer in enumerate(answers):
            answer = answer.strip()
            if answer:
                doc_content = preprocess_vietnamese_text(answer)
                docs.append(Document(page_content=doc_content, metadata={"source": answer_file, "line": i+1}))

# Kiểm tra tài liệu
if not docs:
    raise ValueError("Không có tài liệu nào từ long-a.txt hoặc dang-a.txt. Vui lòng kiểm tra dữ liệu.")

# Chia tài liệu thành các đoạn
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)
split_docs = text_splitter.split_documents(docs)
print(f"Đã chia thành {len(split_docs)} đoạn tài liệu.")

# Khởi tạo và thêm tài liệu vào Chroma vector store
vectorstore = Chroma(
    embedding_function=langchain_embeddings,
    persist_directory="./vnu_HF_db"
)
vectorstore.add_documents(split_docs)
print(f"Đã thêm {len(split_docs)} đoạn tài liệu vào vector store tại ./vnu_HF_db")

Đã di chuyển long-q.txt từ ./data/dataset sang ./data/test
Đã di chuyển long-a.txt từ ./data/dataset sang ./data/test
Đã di chuyển dang-q.txt từ ./data/dataset sang ./data/test
Đã di chuyển dang-a.txt từ ./data/dataset sang ./data/test
Đã chia thành 177 đoạn tài liệu.
Đã thêm 177 đoạn tài liệu vào vector store tại ./vnu_HF_db


# Create traing dataset vector db

In [18]:
import random

# Đọc cặp câu hỏi-trả lời
def read_qa_files(question_file, answer_file):
    qa_pairs = []
    with open(question_file, 'r', encoding='utf-8') as qfile, open(answer_file, 'r', encoding='utf-8') as afile:
        questions = qfile.readlines()
        answers = afile.readlines()
        if len(questions) != len(answers):
            raise ValueError(f"Số dòng không khớp giữa {question_file} và {answer_file}")
        for q, a in zip(questions, answers):
            q = q.strip()
            a = a.strip()
            if q and a:
                qa_pairs.append((q, a))
    return qa_pairs

# Đọc từ ./data/test
test_dir = "./data/test"
qa_pairs = []
qa_pairs.append(read_qa_files(os.path.join(test_dir, "long-q.txt"), os.path.join(test_dir, "long-a.txt")))
qa_pairs.append(read_qa_files(os.path.join(test_dir, "dang-q.txt"), os.path.join(test_dir, "dang-a.txt")))

# Chia dữ liệu: 80% huấn luyện, 20% kiểm tra
random.seed(1024)
test_ratio = 0.2
train_data = []
test_data = []
for qa_pair in qa_pairs:
    random.shuffle(qa_pair)
    split_index = int(len(qa_pair) * (1 - test_ratio))
    train_data.append(qa_pair[:split_index])
    test_data.append(qa_pair[split_index:])

# In số lượng
total_train = sum(len(data) for data in train_data)
total_test = sum(len(data) for data in test_data)
print(f"Số cặp huấn luyện: {total_train}")
print(f"Số cặp kiểm tra: {total_test}")

Số cặp huấn luyện: 141
Số cặp kiểm tra: 36


In [19]:
submit_dir = "./submit/data"
os.makedirs(submit_dir + "/test", exist_ok=True)
os.makedirs(submit_dir + "/train", exist_ok=True)

# Lưu tập kiểm tra
final_test_data = []
for data in test_data:
    final_test_data.extend(data)
test_questions = [q for q, a in final_test_data]
test_answers = [a for q, a in final_test_data]
with open(os.path.join(submit_dir, "test/questions.txt"), "w", encoding="utf-8") as file:
    file.write("\n".join(test_questions))
with open(os.path.join(submit_dir, "test/reference_answers.txt"), "w", encoding="utf-8") as file:
    file.write("\n".join(test_answers))
print(f"Đã lưu {len(test_questions)} câu hỏi và trả lời kiểm tra vào ./submit/data/test")

Đã lưu 36 câu hỏi và trả lời kiểm tra vào ./submit/data/test


In [20]:
final_train_data = []
for data in train_data:
    final_train_data.extend(data)
train_questions = [q for q, a in final_train_data]
train_answers = [a for q, a in final_train_data]
with open(os.path.join(submit_dir, "train/questions.txt"), "w", encoding="utf-8") as file:
    file.write("\n".join(train_questions))
with open(os.path.join(submit_dir, "train/reference_answers.txt"), "w", encoding="utf-8") as file:
    file.write("\n".join(train_answers))
print(f"Đã lưu {len(train_questions)} câu hỏi và trả lời huấn luyện vào ./submit/data/train")

Đã lưu 141 câu hỏi và trả lời huấn luyện vào ./submit/data/train


# Create RAG Chain

In [12]:
# Nhập thư viện
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.prompts import PromptTemplate
import torch
from underthesea import word_tokenize

# Tiền xử lý tiếng Việt
def preprocess_vietnamese_text(text):
    return word_tokenize(text, format="text")

# Khởi tạo embedding
langchain_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
)

# Tải vector store
vectorstore_hf = Chroma(
    embedding_function=langchain_embeddings,
    persist_directory="./vnu_HF_db"
)
vectorstore_qa = Chroma(
    embedding_function=langchain_embeddings,
    persist_directory="./vnu_QA_db"
)

# Khởi tạo model và tokenizer Llama 3.2 3B Instruct
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# Đảm bảo tokenizer có pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Đã tải vector store và khởi tạo mô hình Llama 3.2 3B Instruct.")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Đã tải vector store và khởi tạo mô hình Llama 3.2 3B Instruct.


In [24]:
# Tạo retriever
retriever_hf = vectorstore_hf.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retriever_qa = vectorstore_qa.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Hàm kết hợp retriever
def combined_retriever(query):
    results_hf = retriever_hf.invoke(query)
    results_qa = retriever_qa.invoke(query)
    return results_hf + results_qa

# Hàm định dạng tài liệu
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Tùy chỉnh prompt cho Llama
prompt_template = """
Bạn là một trợ lý AI hữu ích, trả lời câu hỏi bằng tiếng Việt dựa trên thông tin được cung cấp. Hãy trả lời ngắn gọn, chính xác và tự nhiên.

**Thông tin:**
{context}

**Câu hỏi:** {question}

Câu trả lời phải dưới 10 từ
**Trả lời:**
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Hàm sinh câu trả lời với Llama
def generate_answer(question, context, model, tokenizer, max_new_tokens=100):
    formatted_prompt = prompt.format(context=context, question=question)
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Lấy phần trả lời sau "Trả lời:"
    answer = answer.split("Trả lời:")[-1].strip()
    return answer

# Lớp CustomQA
class CustomQA:
    def __init__(self, retriever_fn, model, tokenizer, prompt):
        self.retriever_fn = retriever_fn
        self.model = model
        self.tokenizer = tokenizer
        self.prompt = prompt

    def __call__(self, inputs):
        question = inputs if isinstance(inputs, str) else inputs["query"]
        docs = self.retriever_fn(question)
        context = format_docs(docs)
        answer = generate_answer(question, context, self.model, self.tokenizer)
        return answer

# Tạo hai chuỗi RAG
rag_chain = CustomQA(
    retriever_fn=lambda q: retriever_hf.invoke(q),
    model=model,
    tokenizer=tokenizer,
    prompt=prompt
)

rag_chain_2 = CustomQA(
    retriever_fn=combined_retriever,
    model=model,
    tokenizer=tokenizer,
    prompt=prompt
)

print("Đã tạo hai chuỗi RAG: rag_chain, rag_chain_2.")

Đã tạo hai chuỗi RAG: rag_chain, rag_chain_2.


## answer

In [25]:
# Đọc câu hỏi kiểm tra
def load_questions(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

test_questions_path = "./submit/data/test/questions.txt"
if not os.path.exists(test_questions_path):
    raise FileNotFoundError(f"File {test_questions_path} không tồn tại.")

test_questions = load_questions(test_questions_path)

# Chạy hai chuỗi RAG
answers_rag1 = []
answers_rag2 = []

for question in test_questions:
    processed_question = preprocess_vietnamese_text(question)
    # rag_chain
    answer1 = rag_chain({"query": processed_question}).strip()
    answers_rag1.append(answer1)
    # rag_chain_2
    answer2 = rag_chain_2({"query": processed_question}).strip()
    answers_rag2.append(answer2)

# Lưu câu trả lời
with open("./system_output_rag1.txt", "w", encoding="utf-8") as f:
    for answer in answers_rag1:
        f.write(answer + "\n")
with open("./system_output_rag2.txt", "w", encoding="utf-8") as f:
    for answer in answers_rag2:
        f.write(answer + "\n")

print(f"Đã lưu {len(answers_rag1)} câu trả lời cho rag_chain vào ./system_output_rag1.txt")
print(f"Đã lưu {len(answers_rag2)} câu trả lời cho rag_chain_2 vào ./system_output_rag2.txt")

Đã lưu 36 câu trả lời cho rag_chain vào ./system_output_rag1.txt
Đã lưu 36 câu trả lời cho rag_chain_2 vào ./system_output_rag2.txt


In [26]:
import numpy as np

# Đọc câu trả lời tham chiếu
def load_reference_answers(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return [line.strip().split(";") for line in f if line.strip()]

reference_answers_path = "./submit/data/test/reference_answers.txt"
if not os.path.exists(reference_answers_path):
    raise FileNotFoundError(f"File {reference_answers_path} không tồn tại.")

reference_answers = load_reference_answers(reference_answers_path)

# Đọc câu trả lời từ hai chuỗi
output_files = ["./system_output_rag1.txt", "./system_output_rag2.txt"]
system_answers = {}
for output_file in output_files:
    if not os.path.exists(output_file):
        raise FileNotFoundError(f"File {output_file} không tồn tại.")
    with open(output_file, "r", encoding="utf-8") as f:
        system_answers[output_file] = [line.strip() for line in f if line.strip()]

# Đánh giá
def evaluate_answers(system_answers, reference_answers):
    exact_match = 0
    f1_scores = []
    for sys_ans, refs in zip(system_answers, reference_answers):
        sys_ans = sys_ans.strip()
        refs = [ref.strip() for ref in refs]
        if sys_ans in refs:
            exact_match += 1
        sys_tokens = preprocess_vietnamese_text(sys_ans).lower().split()
        best_f1 = 0
        for ref in refs:
            ref_tokens = preprocess_vietnamese_text(ref).lower().split()
            if not sys_tokens or not ref_tokens:
                continue
            common = len(set(sys_tokens) & set(ref_tokens))
            precision = common / len(sys_tokens) if sys_tokens else 0
            recall = common / len(ref_tokens) if ref_tokens else 0
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
            best_f1 = max(best_f1, f1)
        f1_scores.append(best_f1)
    em_score = exact_match / len(system_answers) if system_answers else 0
    avg_f1 = np.mean(f1_scores) if f1_scores else 0
    return {"exact_match": em_score, "f1_score": avg_f1}

# In kết quả đánh giá
for output_file in output_files:
    evaluation = evaluate_answers(system_answers[output_file], reference_answers)
    print(f"\nĐánh giá cho {output_file}:")
    print(f"Exact Match: {evaluation['exact_match']:.4f}")
    print(f"F1 Score: {evaluation['f1_score']:.4f}")


Đánh giá cho ./system_output_rag1.txt:
Exact Match: 0.0000
F1 Score: 0.0715

Đánh giá cho ./system_output_rag2.txt:
Exact Match: 0.0000
F1 Score: 0.0874
